In [1]:
import cv2
import time
import pandas as pd
import threading
from datetime import datetime,timedelta

# Read the Excel file
video_list_df = pd.read_excel('Video_List.xlsx')

# Get the list of URLs and corresponding SigID from the 'IPAdd' and 'SigID' columns
urls = video_list_df['IPAdd'].tolist()
sig_ids = video_list_df['SigID'].tolist()

# Set start and end time
start_time = datetime(2024, 5, 29, 12, 15, 0)
end_time = datetime(2024, 5, 29, 23, 59, 00)

# Set start time to current system time
#start_time = datetime.now()

# Set end time to 60 seconds after the start time
#end_time = start_time + timedelta(seconds=60)

# Global variable to stop recording
stop_recording = False

def record_video(udp_url, sig_id, start_time, end_time):
    global stop_recording

    # Wait until the start time
    while datetime.now() < start_time:
        time.sleep(0.01)  # Sleep a little to prevent busy waiting

    # Open the video stream
    cap = cv2.VideoCapture(udp_url)

    # Check if the video stream is opened successfully
    if not cap.isOpened():
        print(f"Error: Could not open the video stream from {udp_url}.")
        return

    # Get the original resolution of the video stream
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    output_filename = f'{sig_id}.avi'
    out = cv2.VideoWriter(output_filename, fourcc, 30.0, (frame_width, frame_height))

    while not stop_recording:
        ret, frame = cap.read()
        if ret:
            # Add timestamp to the frame
            current_time = datetime.now()
            timestamp = current_time.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]

            # Get the width and height of the frame
            frame_height, frame_width = frame.shape[:2]

            # Set the position of the timestamp at the bottom center of the frame
            text_size, _ = cv2.getTextSize(timestamp, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
            text_x = (frame_width - text_size[0]) // 2
            text_y = frame_height - 10

            # Add the timestamp with green color
            cv2.putText(frame, timestamp, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            # Write the frame to the output file
            out.write(frame)

            # Display the frame (optional)
            # cv2.imshow('Frame', frame)

            # Break the loop if the end time has been reached
            if datetime.now() >= end_time:
                stop_recording = True
        else:
            break

    # Release everything if job is finished
    cap.release()
    out.release()
    # cv2.destroyAllWindows()

# Create and start a thread for each video
threads = []
for udp_url, sig_id in zip(urls, sig_ids):
    thread = threading.Thread(target=record_video, args=(udp_url, sig_id, start_time, end_time))
    thread.start()
    threads.append(thread)

# Wait for all threads to finish
for thread in threads:
    thread.join()

cv2.destroyAllWindows()
